In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import tensorboard
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
from sklearn.model_selection import KFold

Using cuda device


In [2]:
transform = transforms.Compose(
            [transforms.Resize([32, 32]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [4]:
train_dataset = datasets.ImageFolder(root='D:/split_data_breast/train', 
                                     transform=transform)
test_dataset  = datasets.ImageFolder(root='D:/split_data_breast/test',  
                                     transform=transform)
valid_dataset  = datasets.ImageFolder(root='D:/split_data_breast/val',  
                                     transform=transform)

In [5]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 222018
    Root location: D:/split_data_breast/train
    StandardTransform
Transform: Compose(
               Resize(size=[32, 32], interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [6]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 27755
    Root location: D:/split_data_breast/test
    StandardTransform
Transform: Compose(
               Resize(size=[32, 32], interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [7]:
valid_dataset

Dataset ImageFolder
    Number of datapoints: 27751
    Root location: D:/split_data_breast/val
    StandardTransform
Transform: Compose(
               Resize(size=[32, 32], interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [23]:
dataiter = iter(train_dataloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 3, 32, 32])
torch.Size([64])


In [24]:
import torch
import torch.nn as nn
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
#squeezenet = models.squeezenet1_0(pretrained=True)
densenet = models.densenet161(pretrained=True)
#inception = models.inception_v3()

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [27]:
for param in resnet18.parameters():
    param.requires_grad = False
    
resnet18.fc = nn.Sequential(nn.Linear(512, 2))

In [28]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.to(device)
    size = len(dataloader)
    for i, (X, y) in enumerate(dataloader):
        if torch.cuda.is_available():
            
            #X = X.reshape(1, 3, 224, 224)
            #X = X.float()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

        # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                loss, current = loss.item(), i * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.to(device)
    size = len(dataloader)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    total =0

    with torch.no_grad():
        
        for i, (X, y) in enumerate(dataloader):
            if torch.cuda.is_available():
                
                #X = X.reshape(1, 3, 224, 224)
                #X = X.float()
                X, y = X.to(device), y.to(device)
                pred = model(X)
                total+= y.size(0)
                
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    
    test_loss /= num_batches
    correct /= total
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [29]:
import torch.optim as optim
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.01)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader,resnet18, loss_fn, optimizer)
    test_loop(test_dataloader, resnet18, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.044055  [    0/ 3470]
loss: 0.472728  [ 6400/ 3470]
loss: 0.478305  [12800/ 3470]
loss: 0.607661  [19200/ 3470]
loss: 0.441085  [25600/ 3470]
loss: 0.444096  [32000/ 3470]
loss: 0.535683  [38400/ 3470]
loss: 0.381679  [44800/ 3470]
loss: 0.614595  [51200/ 3470]
loss: 0.421297  [57600/ 3470]
loss: 0.547027  [64000/ 3470]
loss: 0.410666  [70400/ 3470]
loss: 0.452843  [76800/ 3470]
loss: 0.655014  [83200/ 3470]
loss: 0.365226  [89600/ 3470]
loss: 0.611859  [96000/ 3470]
loss: 0.409308  [102400/ 3470]
loss: 0.570176  [108800/ 3470]
loss: 0.403921  [115200/ 3470]
loss: 0.454883  [121600/ 3470]
loss: 0.364330  [128000/ 3470]
loss: 0.357529  [134400/ 3470]
loss: 0.356735  [140800/ 3470]
loss: 0.468400  [147200/ 3470]
loss: 0.372672  [153600/ 3470]
loss: 0.438929  [160000/ 3470]
loss: 0.384476  [166400/ 3470]
loss: 0.390592  [172800/ 3470]
loss: 0.426675  [179200/ 3470]
loss: 0.315214  [185600/ 3470]
loss: 0.418336  [192000/ 3470]
loss: 0.596019 